In [1]:
# from pyspark.sql import SparkSession

# # Spark session & context
# spark = (SparkSession
#          .builder
#          .master("local")
#          .appName("load-postgres")
#          # Add postgres jar
#          .config("spark.driver.extraClassPath", "/home/sengopal/work/jars/postgresql-9.4.1207.jar")
#          .getOrCreate())
# sc = spark.sparkContext

# df.select(df.corpus, F.struct(df.word_count, df.filtered.getItem(0).alias('word')).alias('count_word')) \
#   .where(F.col('count_word').getItem('word').isNotNull()) \
#   .groupby('corpus') \
#   .agg({'count_word': 'max'}) \
#   .orderBy('corpus') \
#   .select(
#      'corpus',
#      F.col('max(count_word)').getItem('word').alias('word'),
#      F.col('max(count_word)').getItem('word_count').alias('count')) \
#   .toPandas()


# from pyspark.sql.functions import explode, split, regexp_replace
# # explode(split(  # \'\[\]
# df2 = df1.select(df.name,F.explode(F.split(F.regexp_replace(df.id_artists, "['\[\] ]", ""),",")))
# # df2.printSchema()
# df2.toPandas()


# from pyspark.sql.functions import count, when, isnan, col
# df_tracks.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
import pandas as pd
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_2025/1303964058.py:39: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master('local').appName('bq').config('spark.driver.extraClassPath', 'spark-bigquery-with-dependencies_2.12-0.23.2.jar').getOrCreate()
# Read the data in from BigQuery
# Not necessary as the file seems to be correctly detected from the env variables
# spark.conf.set("credentialsFile", "/.google/credentials/google_credentials.json ")

In [3]:
df_tracks = spark.read \
  .format('bigquery') \
  .load('fluent-tea-338517.spotify_monthly_data.tracks_data')

df_artists = spark.read \
  .format('bigquery') \
  .load('fluent-tea-338517.spotify_monthly_data.artists_data')

In [4]:
### tracks data preparation
df_tracks = df_tracks.withColumnRenamed("id","track_id").withColumnRenamed("name","track_name").withColumnRenamed("popularity","track_popularity")

### artists data preparation
df_artists = df_artists.withColumnRenamed("id","artist_id").withColumnRenamed("name","artist_name").withColumnRenamed("popularity","artist_popularity")

In [5]:
# Fix the tracks to explode for the artist_id
df_tracks = df_tracks.withColumn('cleaned_artist_id', F.explode(F.split(F.regexp_replace('id_artists', "['\[\] ]", ""),",")))

# Selecting only the exploded columns and the other required columns
df_tracks = df_tracks.select([c for c in df_tracks.columns if c not in {'artists', 'id_artists'}])

In [6]:
# tracks_artists_data
df_tracks = df_tracks.alias('df_tracks')
df_artists = df_artists.alias('df_artists')

df_tracks_artists_data = df_tracks.join(df_artists, df_tracks.cleaned_artist_id == df_artists.artist_id).select('df_tracks.*', 'df_artists.*')
df_tracks_artists_data = df_tracks_artists_data.select([c for c in df_tracks_artists_data.columns if c not in {'cleaned_artist_id'}])
df_tracks_artists_data.printSchema()
df_tracks_artists_data.show(5)

root
 |-- track_id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_popularity: long (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- explicit: long (nullable = true)
 |-- release_date: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: long (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: long (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- followers: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_popularity: long (nullable = true)

+--------------------+--------------------+------

In [7]:
# Formulating the final table
# Saving the data to BigQuery
# df_tracks_artists_data.write.format('bigquery') \
#   .option("temporaryGcsBucket","dtc_data_lake_fluent-tea-338517") \
#   .option('table', 'fluent-tea-338517.spotify_monthly_data.tracks_artists_data') \
#   .save()



In [15]:
print(df_tracks_artists_data.count())
print(df_test.count())

730946
730946


In [11]:
df_tracks_artists_data.toPandas().isnull().sum()

track_id             0
track_name           0
track_popularity     0
duration_ms          0
explicit             0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
artist_id            0
followers            2
genres               0
artist_name          0
artist_popularity    0
dtype: int64

In [13]:
df_tracks_artists_data = df_tracks_artists_data.fillna( { 'followers':0})
df_tracks_artists_data.toPandas().isna().sum()

track_id             0
track_name           0
track_popularity     0
duration_ms          0
explicit             0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
artist_id            0
followers            0
genres               0
artist_name          0
artist_popularity    0
dtype: int64

In [4]:
df_test = df_tracks.limit(5)

In [5]:
df_test.toPandas().to_csv('test.csv',index=False,header=False)